In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# データ読み込み

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import warnings
warnings.simplefilter('ignore')
pd.options.display.max_rows=1000
pd.options.display.max_columns=500

stock_list = pd.read_csv('/kaggle/input/jpx-tokyo-stock-exchange-prediction/stock_list.csv', parse_dates=['EffectiveDate'])
financials = pd.read_csv('/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/financials.csv', parse_dates=['Date', 'DisclosedDate', 'CurrentPeriodEndDate', 'TypeOfCurrentPeriod', 'CurrentFiscalYearStartDate', 'CurrentFiscalYearEndDate'])
stock_prices = pd.read_csv('/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv', parse_dates=['Date'])

stock_fin_spec = pd.read_csv('/kaggle/input/jpx-tokyo-stock-exchange-prediction/data_specifications/stock_fin_spec.csv')
stock_list_spec = pd.read_csv('/kaggle/input/jpx-tokyo-stock-exchange-prediction/data_specifications/stock_list_spec.csv')
stock_price_spec = pd.read_csv('/kaggle/input/jpx-tokyo-stock-exchange-prediction/data_specifications/stock_price_spec.csv')

# 変数の説明

In [ ]:
data_specifications = {'stock_price_spec': stock_price_spec, 'stock_list_spec': stock_list_spec, 'stock_fin_spec': stock_fin_spec}
for name, data in data_specifications.items():
    print(name)
    print(data.shape)
    display(data)
    print('\n\n')

# Stock_prices

In [ ]:
display(stock_prices.head())
print(stock_prices.shape)

In [ ]:
stock_prices.info()

In [ ]:
# 統計量確認
display(stock_prices.describe())
display(stock_prices.describe(include=['O']))

In [ ]:
# ブール値を変換
stock_prices = stock_prices.replace({True: 1, False: 0})

In [ ]:
# 監査の有無でデータの特徴を見る
print(stock_prices['SupervisionFlag'].value_counts())
print(stock_prices.groupby('SupervisionFlag')['Target'].mean())
print(stock_prices.groupby('SupervisionFlag')['Target'].std())
print(stock_prices.groupby('SupervisionFlag')['Volume'].mean()/1000)
print(stock_prices.groupby('SupervisionFlag')['ExpectedDividend'].mean())

In [ ]:
# 欠損値の確認
# ExpectedDividendはほぼ欠損
stock_prices.isnull().sum()

In [ ]:
# OHLCの欠損は取引がなかったことを示す
len(stock_prices[stock_prices['Volume']==0]) == stock_prices['Open'].isnull().sum()

In [ ]:
# 収録期間
# 2017年1月4日～2021年12月3日
stock_prices['Date'] = stock_prices['Date'].dt.round('D')
start = stock_prices['Date'].min()
end = stock_prices['Date'].max()
print(start)
print(end)
print(f'{end-start}日')

In [ ]:
# ユニーク数
stock_prices.nunique()

# Stock_list

In [ ]:
# データの先頭と型
display(stock_list.head())
print(stock_list.shape)

In [ ]:
# データ情報
stock_list.info()

In [ ]:
# ユニーク数
stock_list.nunique()

In [ ]:
# 統計量確認
display(stock_list.describe())
display(stock_list.describe(include=['O']))

In [ ]:
# 証券コードに重複なし
print(stock_list['SecuritiesCode'].nunique())
len(stock_list)

In [ ]:
# Nameの重複部分を確認
stock_list[stock_list['Name'].duplicated(keep=False)]

In [ ]:
# 業種名の確認
print(stock_list['33SectorName'].unique())
len(stock_list['33SectorName'].unique())

In [ ]:
# 業種名が「-」となっているもの
print(len(stock_list.loc[stock_list['33SectorName']=='-']))
print(len(stock_list.loc[(stock_list['33SectorName']=='-') & (stock_list['NewMarketSegment'].isnull())]))
# 500社のうち490社が新市場区分が欠損となっている
#print(stock_list.loc[stock_list['33SectorName']=='-']['NewMarketSegment'].unique())

In [ ]:
# 業種名無しで新市場区分が欠損でないもの
stock_list.loc[(stock_list['33SectorName']=='-') & (~stock_list['NewMarketSegment'].isnull())]

In [ ]:
# 株価リストには含まれていない
stock_list_sectorname_nan = set(stock_list.loc[(stock_list['33SectorName']=='-')])
stock_prices_sec_unique = set(stock_prices['SecuritiesCode'].unique())
stock_list_sectorname_nan & stock_prices_sec_unique

In [ ]:
stock_list['33SectorName'].value_counts()

In [ ]:
stock_list['EffectiveDate'].unique()

In [ ]:
stock_list_2021 = stock_list.loc[stock_list['EffectiveDate'].dt.strftime('%Y')=='2021']
print(len(stock_list_2021))
print(stock_list_2021['SecuritiesCode'].nunique())

In [ ]:
# 2021年末における時価総額(億円)
(stock_list_2021['MarketCapitalization']/100000000).describe()

In [ ]:
stock_list_2021['MarketCapitalization']

# financials

In [ ]:
# 財務データの先頭5行と形状
display(financials.head())
print(financials.shape)

In [ ]:
financials.info()

In [ ]:
 # 型
int_list = [
    'DisclosureNumber', 'SecuritiesCode','DisclosedUnixTime', 'NetSales', 'OperatingProfit', 'OrdinaryProfit', 'Profit', 
    'TotalAssets', 'Equity','ForecastNetSales', 'ForecastOperatingProfit', 'ForecastOrdinaryProfit', 'ForecastProfit',
    'NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock', 'NumberOfTreasuryStockAtTheEndOfFiscalYear', 'AverageNumberOfShares',
    'ApplyingOfSpecificAccountingOfTheQuarterlyFinancialStatements', 'MaterialChangesInSubsidiaries', 'ChangesBasedOnRevisionsOfAccountingStandard',
    'ChangesOtherThanOnesBasedOnRevisionsOfAccountingStandard', 'ChangesInAccountingEstimates', 'RetrospectiveRestatement'
    ]
float_list = [
    'EarningsPerShare', 'EquityToAssetRatio', 'BookValuePerShare', 'ResultDividendPerShare1stQuarter', 'ResultDividendPerShare2ndQuarter',
    'ResultDividendPerShare3rdQuarter', 'ResultDividendPerShareFiscalYearEnd', 'ResultDividendPerShareAnnual','ForecastDividendPerShare1stQuarter',
    'ForecastDividendPerShare2ndQuarter', 'ForecastDividendPerShare3rdQuarter', 'ForecastDividendPerShareFiscalYearEnd','ForecastDividendPerShareAnnual',
    'ForecastEarningsPerShare'
    ]

In [ ]:
# 統計量確認
display(financials.describe())
display(financials.describe(include=['O']))

In [ ]:
# ユニーク数
print(len(financials))
financials.nunique()

In [ ]:
# 欠損値の確認
financials.isnull().sum()

In [ ]:
financials['TypeOfCurrentPeriod'].value_counts()
# 四半期ごと + 決算月
# 4Q, 5Qは稀

In [ ]:
financials['MaterialChangesInSubsidiaries'].value_counts()

In [ ]:
financials['ChangesBasedOnRevisionsOfAccountingStandard'].value_counts()

In [ ]:
financials['ChangesInAccountingEstimates'].value_counts()

In [ ]:
financials['ChangesOtherThanOnesBasedOnRevisionsOfAccountingStandard'].value_counts()

# データセットの結合

In [ ]:
stock_list_2 = stock_list[['SecuritiesCode', 'Name', '33SectorName', '17SectorName', 'NewMarketSegment', 'NewIndexSeriesSize']]

In [ ]:
# 主キーが一致しているかどうか確認
def check_pk(left_df, right_df, key):
    left_key = left_df[key].unique()
    right_key = right_df[key].unique()
    if len(left_key) == len(set(left_key)&set(right_key)):
        if left_df[key].nunique()==right_df[key].nunique():
            print("PKのユニーク数は一致しています。")  
        else:
            print("PKのユニーク数は一致していません")
    else:
        print("両方のデータフレームでPKのユニークが一致していません。どちらかにしか含まれていない値が存在しています。")
    return

check_pk(stock_prices, stock_list, 'SecuritiesCode')

# stock_pricesの証券コードがすべてstock_listの証券コードにあるかどうか
(set(stock_prices['SecuritiesCode']) & set(stock_list_2['SecuritiesCode'])) == set(stock_prices['SecuritiesCode'])

In [ ]:
# 収益率の平均と分散
stock_list_2 = stock_list[['SecuritiesCode', 'Name', '33SectorName', 'Section/Products', '17SectorName', 'NewMarketSegment', 'NewIndexSeriesSize']]
target_std = stock_prices.groupby('SecuritiesCode')['Target'].std().rename('TargetStd')
target_mean = stock_prices.groupby('SecuritiesCode')['Target'].mean().rename('TargetMean')
stocks = pd.merge(stock_list_2, target_std, on='SecuritiesCode')
stocks = pd.merge(stocks, target_mean, on='SecuritiesCode')
stocks['TargetStdRank'] = stocks['TargetStd'].rank(axis=0, ascending=True).astype(int)
display(stocks.sort_values('TargetStdRank', ascending=True).head())
display(stocks.sort_values('TargetStdRank', ascending=True).tail())
display(stocks.sort_values('TargetMean', ascending=True).head())
display(stocks.sort_values('TargetMean', ascending=True).tail())

In [ ]:
# IT・サービス業はリターンのばらつきが大きい
stocks.groupby('17SectorName')['TargetStd'].mean().sort_values(ascending=False)

In [ ]:
# stock_pricesとfinancialsを結合

# 可視化

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 業種ごとの上場企業数
plt.figure(figsize=(15, 6))
plt.bar(x=stocks['33SectorName'].value_counts().index , height = stocks['33SectorName'].value_counts().values)
plt.xticks(rotation= 90)
plt.show()

In [ ]:
# 業種ごとのTarget平均
plt.figure(figsize=(15, 6))
plt.bar(x=stocks.groupby('33SectorName')['TargetMean'].mean().sort_values(ascending=False).index, height=stocks.groupby('33SectorName')['TargetMean'].mean().sort_values(ascending=False).values)
plt.xticks(rotation= 90)
plt.show()

In [ ]:
# 業種ごとのTarget標準偏差
plt.figure(figsize=(15, 6))
plt.bar(x=stocks.groupby('33SectorName')['TargetStd'].mean().sort_values(ascending=False).index, height=stocks.groupby('33SectorName')['TargetStd'].mean().sort_values(ascending=False).values)
plt.xticks(rotation= 90)
plt.show()

In [ ]:
# 旧市場別と新市場別企業数
fig, axes = plt.subplots(1, 2, figsize=(20, 10))
axes[0].bar(x=stocks['Section/Products'].value_counts().index, height=stocks['Section/Products'].value_counts().values)
axes[1].bar(x=stocks['NewMarketSegment'].value_counts().index, height=stocks['NewMarketSegment'].value_counts().values)
plt.show()

In [ ]:
# 2021時価総額分布
# 外れ値が多い
plt.figure(figsize=(8, 8))
plt.boxplot(stock_list_2021['MarketCapitalization']/100000000)
plt.show()

In [ ]:
# 業種別時価総額平均(単位: 億円)
x = stock_list_2021.groupby('33SectorName')['MarketCapitalization'].mean().sort_values(ascending=False).index
y = stock_list_2021.groupby('33SectorName')['MarketCapitalization'].mean().sort_values(ascending=False).values/100000000
def barplot(x, y):
    plt.figure(figsize=(10, 6))
    plt.bar(x, y)
    plt.xticks(rotation= 90)
    plt.show()
    
barplot(x, y)

In [ ]:
# 日毎の総取引量と終値平均の推移
fig = plt.figure(figsize=(20, 10))
ax1 = fig.add_subplot(2, 1, 1)
ax1.plot(stock_prices.groupby('Date')['Close'].mean().index, stock_prices.groupby('Date')['Close'].mean().values)
ax1.set_xlabel('Date')
ax1.set_ylabel('Close')
ax2 = fig.add_subplot(2, 1, 2)
ax2.bar(stock_prices.groupby('Date')['Volume'].sum().index, stock_prices.groupby('Date')['Volume'].sum().values)
ax2.set_xlabel('Date')
ax2.set_ylabel('Volume')
plt.show()

In [ ]:
# Targetの平均の推移
plt.figure(figsize=(12, 5))
plt.plot(stock_prices.groupby('Date')['Target'].mean().index, stock_prices.groupby('Date')['Target'].mean().values)
plt.show()

In [ ]:
sns.histplot(stock_prices.groupby('Date')['Target'].mean().values)
plt.show()

In [ ]:
# Targetの分布の一例
plt.figure(figsize=(15, 5))
plt.xticks(rotation=90)
sns.lineplot(x=stock_prices.loc[stock_prices['SecuritiesCode']==1301]['Date'], y=stock_prices.loc[stock_prices['SecuritiesCode']==1301]['Target'])
plt.show()

In [ ]:
# Targetのヒストグラムの一例
# 正規分布に近い
sns.histplot(stock_prices.loc[stock_prices['SecuritiesCode']==4716]['Target'])
plt.show()

In [ ]:
# 検定により確認
from scipy import stats
import numpy as np
bins, bin_edges = np.histogram(stock_prices.loc[stock_prices['SecuritiesCode']==4716]['Target'], bins='auto')
stat, p = stats.shapiro(bins)
print(f'シャピロウィルク検定のp値: {p}')

In [ ]:
# 移動平均線の一例(2021年)
code = np.random.choice(stock_prices['SecuritiesCode'].unique(), 1)[0]
example = stock_prices.loc[(stock_prices['SecuritiesCode']==code) & (stock_prices['Date'] > '2020-12-31')]
plt.figure(figsize=(15, 5))
sns.lineplot(x=example.Date, y=example.Close, label='stock price')
sns.lineplot(x=example.Date, y=example.Close.rolling(5).mean(), label='SMA 5')
sns.lineplot(x=example.Date, y=example.Close.rolling(25).mean(), label='SMA 25')
sns.lineplot(x=example.Date, y=example.Close.rolling(75).mean(), label='SMA 75')
plt.title(f'code: {code}')
plt.show()

In [ ]:
# 相関係数
plt.figure(figsize=(10,10))
sns.heatmap(stock_prices.corr(), cmap=sns.color_palette('coolwarm', 10), annot=True, fmt='.2f', vmin=-1, vmax=1)
plt.show()

In [ ]:
# 累積リターンをランダムに1銘柄表示
stock_prices['R'] = 1 + stock_prices.groupby('SecuritiesCode')['Close'].pct_change()
stock_prices['cum_return'] = stock_prices.groupby('SecuritiesCode')['R'].cumprod()
code = np.random.choice(stock_prices['SecuritiesCode'].unique(), 1)
plt.figure(figsize=(10, 8))
sns.lineplot(x=stock_prices.loc[stock_prices['SecuritiesCode']==code[0]]['Date'], y=stock_prices.loc[stock_prices['SecuritiesCode']==code[0]]['cum_return'])
plt.xlabel('Date')
plt.title(f'Cum_Return: CODE {code[0]}')
plt.show()